## Stable Moving CA

### Imports

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision 
from tqdm import tqdm

import IPython
from IPython.display import display, HTML

In [8]:
#device = torch.device('mps:0' if torch.backends.mps.is_available else 'cpu')
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')
#device = torch.device('cpu')
print(device)

cuda:0


### Model

In [9]:
class CA(nn.Module):
    def __init__(self):
        super(CA, self).__init__()
        self.conv1 = nn.Conv2d(2, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 8, 1, padding=0)
        self.conv3 = nn.Conv2d(8, 2, 1, padding=0)

    def forward(self, x):
        print(x.shape)
        z = self.conv1(x)
        print(z.shape)
        z = torch.relu(z)
        print(z.shape)
        z = self.conv2(z)
        print(z.shape)
        z = torch.relu(z)
        print(z.shape)
        z = self.conv3(z)
        #z = torch.sigmoid(z)

        #trying to ensure 
        #scale = torch.sum(x) / torch.sum(z)
        #z = torch.mul(z, scale)

        #trying to ensure the same amount of alive CAs
        #alive_count = torch.count_nonzero(torch.where(z > 0.1, z, 0))
        #return z, alive_count
        print(z.shape)
        return z

    #add helper functions

### Training

In [10]:
def train(model, state, iterations, optimizer, criterion, epochs=1):
    #mat = torch.rand(32, 32, device=device)
    mat = torch.zeros(32, 32, device=device)
    mat2 = torch.rand(3,3, device=device)
    mat[:3, :3] = mat2
    mat[-3:, -3:] = mat2
    #state = np.stack([1 - state, state])
    #state = torch.stack([1 - state, state])
    state = torch.stack([1 - mat, mat])
    #state = torch.rand(2, 32, 32, device=device)
    losses = np.zeros(iterations * epochs)
    #for epoch in range(epochs):
    for epoch in tqdm(range(epochs)):
        #for i in tqdm(range(iterations)):
        for i in range(iterations):
            #input_alive_count = torch.count_nonzero(torch.where(state > 0.1, state, 0)).type(torch.float32)
            #torch.where(x > 0 | x<0.1, x, y)
            #kunne lave en tensor med count på mellem 0.0-0.1, 0.1-0.2, 0.2-0.3 ...
            optimizer.zero_grad()
            state_hat = state
            for j in range(epochs):
                #state_hat, alive_count = model(state_hat)
                print(state_hat.shape)
                state_hat = model(state_hat)
                if j + 1 < epoch:
                    loss = criterion(state_hat, state)
                    loss.backward(retain_graph=True)

            #scale = torch.sum(state) / torch.sum(state_hat)
            loss = criterion(state_hat, state)
            #loss.requires_grad = True
            #loss2 = criterion(scale, torch.tensor([1.0], device=device))
            #alive_count = alive_count.type(torch.float32)
            #loss3 = criterion(alive_count, input_alive_count) * 0.000001
            #loss = loss1 #+ loss2 #+ loss3
            losses[epoch*iterations + i] = loss.item()
            loss.backward()
            optimizer.step()
        print(loss)
    return model, losses

### Main

In [ ]:
#initial state
#...
glider = np.array([[0,1,0],
                   [0,0,1],
                   [1,1,1]])
state = np.zeros((32, 32), dtype=np.float32)
state[:3,:3] = glider
state = np.stack([1 - state, state]) # why? - what does this do
state = torch.from_numpy(state)
state = state.to(device)

model = CA().to(device)
iterations = 400
epochs = 20
lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr=lr)
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
model, losses = train(model, state, iterations, optimizer, criterion, epochs)

In [ ]:
#plot
plt.plot(losses)

NameError: name 'losses' is not defined

In [ ]:
#save progress
torch.save(model.state_dict(), 'model1.pth')

### Test - Animate

In [ ]:
model = CA()
model.load_state_dict(torch.load('model.pth'))
model = model.to(device)
#Setup
glider = np.array([[0,1,0],
                   [0,0,1],
                   [1,1,1]])
state = np.zeros((32, 32))
state[:3,:3] = glider
#state[0,0]= 1
state = np.stack([1 - state, state])
state = torch.from_numpy(state).float().unsqueeze(0).to(device)
print(state.shape)

In [ ]:
#Test with random state
mat = torch.rand(32, 32, device=device)
state = torch.stack([1 - mat, mat]).unsqueeze(0)
print(state.shape)

In [ ]:
#iterate
@torch.no_grad()
def animate(i):
    global state
    #state = torch.softmax(model(state), dim=1).to(device) #calculate new state
    state = model(state) #calculate new state
    mat.set_data(state.detach().cpu().numpy()[0,0]) #update the mat shown to reflect the new state

fig, ax = plt.subplots()
mat = ax.matshow(state.detach().cpu().numpy()[0,0], cmap="gray") #show mat in fig

IPython.display.HTML(animation.FuncAnimation(fig, animate, frames=1000, interval=10).to_jshtml())

In [ ]:
state1 = state
state1 = model(state1)
plt.imshow(state1.reshape(2, 32,32).cpu().detach().numpy()[0], cmap='gray')

In [ ]:
print(model)

In [ ]:
for p in model.parameters():
    print(p)